In [1]:
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

DAY = 9
YEAR = 2022

In [2]:
# use real data
raw = get_data(day=DAY, year=YEAR)

# print(raw)

In [3]:
direction = {
    "R": [1, 0],
    "L": [-1, 0],
    "U": [0, 1],
    "D": [0, -1],
}


def parse_data(data):
    data = data.split("\n")
    data = [d.split(" ") for d in data]
    data = [(direction[d[0]], int(d[1])) for d in data]
    return data


data = parse_data(raw)

# Part 1

In [4]:
positions = set()
tpoz = np.array([0, 0])
hpoz = np.array([0, 0])
positions.add(tuple(np.array([0, 0])))


def move_tail(hpoz, tpoz):
    diff = hpoz - tpoz
    if np.all(abs(diff) <= 1):
        return tpoz

    tpoz += np.clip(diff, -1, 1)
    return tpoz


for move, dist in data:
    for s in range(dist):
        hpoz += move
        tpoz = move_tail(hpoz, tpoz)
        positions.add(tuple(tpoz))

result = len(positions)
result

6026

In [5]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [6]:
positions = set()
rope = np.zeros((10, 2))
positions.add(tuple(rope[-1]))

for move, dist in data:
    for s in range(dist):
        rope[0] += move  # move head
        for idx, knot in enumerate(rope[1:]):
            rope[idx + 1] = move_tail(rope[idx], knot)

        positions.add(tuple(rope[-1]))

result = len(positions)
result

2273

In [7]:
# submit(result, part="b", day=DAY, year=YEAR)

# Animation

In [ ]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

multiprocessing.set_start_method("fork")

In [8]:
gif = []
rope = np.zeros((10, 2))
gif.append(rope.copy().astype(int))

for tdx, (move, dist) in enumerate(data):
    for s in range(dist):
        rope[0] += move  # move head
        for idx, knot in enumerate(rope[1:]):
            rope[idx + 1] = move_tail(rope[idx], knot)

        gif.append(rope.copy().astype(int))

In [21]:
minx, miny = np.min(np.concatenate(gif), axis=0)
maxx, maxy = np.max(np.concatenate(gif), axis=0)

n, m = int(maxx - minx) + 1, int(maxy - miny) + 1

images = []
mask = np.zeros((n, m, 3))
for rope in gif:
    current = np.zeros((n, m, 3))
    current[rope.T[0] - minx, rope.T[1] - miny] = [255, 255, 255]
    mask[rope[[-1]].T[0] - minx, rope[[-1]].T[1] - miny] = [47, 47, 1]

    images.append(np.clip((mask + current).copy(), 0, 255).astype(int))

In [24]:
def plot_scene(args):
    idx, img = args
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(img)
    ax.axis("off")
    plt.savefig(f"figs/{idx:05}.png", bbox_inches="tight", transparent=False)
    plt.close()

In [25]:
!rm -rf './figs/*'

step = 10
to_plot = list(enumerate(images[::step]))

with ProcessPoolExecutor() as executor:
    list(tqdm(executor.map(plot_scene, to_plot), total=len(to_plot)))

  0%|          | 0/1133 [00:00<?, ?it/s]

In [26]:
!ffmpeg -f image2 -framerate 15 -i figs/%05d.png -vf "split[s0][s1];[s0]palettegen=stats_mode=diff[p];[s1][p]paletteuse=dither=bayer:bayer_scale=5:diff_mode=rectangle" out.gif -y

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.102)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
